<a href="https://colab.research.google.com/github/marschneatcisco/python-notes/blob/main/pandas_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

url = (
    "https://raw.github.com/pandas-dev"
    "/pandas/main/pandas/tests/io/data/csv/tips.csv"
)


tips = pd.read_csv(url)

# Another dataframes used later:
frame = pd.DataFrame(
    {"col1": ["A", "B", np.NaN, "C", "D"], "col2": ["F", np.NaN, "G", "H", "I"]}
)

df1 = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})

df2 = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})

In [ ]:
tips

# **SELECT**

In [ ]:
# ORDER BY "tip" column

tips.sort_values("tip")


In [ ]:
# SELECT total_bill, tip, smoker, time
# FROM tips;

tips[["total_bill", "tip", "smoker", "time"]]


In [ ]:
# SELECT *, tip/total_bill as tip_rate
# FROM tips;

tips.assign(tip_rate=tips["tip"] / tips["total_bill"])

# **WHERE**

In [ ]:
# SELECT *
# FROM tips
# WHERE time = 'Dinner';

# This is simply passing a Series of True/False objects to the DataFrame, returning all rows with True.

tips[tips["total_bill"] > 10]

In [19]:
# Another way:

is_dinner = tips["time"] == "Dinner"
is_dinner.value_counts()


True     176
False     68
Name: time, dtype: int64

In [ ]:
tips[is_dinner]


In [ ]:
# SELECT *
# FROM tips
# WHERE time = 'Dinner' AND tip > 5.00;

tips[(tips["time"] == "Dinner") & (tips["tip"] > 5.00)]

In [ ]:
# SELECT *
# FROM tips
# WHERE size >= 5 OR total_bill > 45;

tips[(tips["size"] >= 5) | (tips["total_bill"] > 45)]

In [50]:
# Is Null checking

frame = pd.DataFrame(
    {"col1": ["A", "B", np.NaN, "C", "D"], "col2": ["F", np.NaN, "G", "H", "I"]}
)

frame

,col1,col2
0,A,F
1,B,NaN
2,NaN,G
3,C,H
4,D,I


In [ ]:
# SELECT *
# FROM frame
# WHERE col2 IS NULL;

frame[frame["col2"].isna()]

In [ ]:
# SELECT *
# FROM frame
# WHERE col1 IS NOT NULL;

frame[frame["col1"].notna()]

# COUNT

In [52]:
# Counts the non-NaN values
frame["col1"].count()

4

In [56]:
# Counts the number of rows, regardless of null or not
frame["col1"].size


10

# **GROUP BY**

In [26]:
# SELECT sex, count(*)
# FROM tips
# GROUP BY sex;

tips.groupby("sex").size()

# Notice the use of size() and not count(). This is because count() applies 
# the function to each column, returning the number of NOT NULL records within 
# each.

sex
Female     87
Male      157
dtype: int64

In [ ]:
# SELECT day, AVG(tip), COUNT(*)
# FROM tips
# GROUP BY day;

tips.groupby("day").agg({"tip": np.mean, "day": np.size})
# using some NumPy math functions

In [ ]:
# SELECT smoker, day, COUNT(*), AVG(tip)
# FROM tips
# GROUP BY smoker, day;
tips.groupby(["smoker", "day"]).agg({"tip": [np.size, np.mean]})

# **JOIN**

In [3]:
df1 = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})

df2 = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})

In [4]:
df1

,key,value
0,A,-1.908681
1,B,0.427489
2,C,0.352510
3,D,-0.455809


In [34]:
df2

,key,value
0,B,1.246087
1,D,0.267525
2,D,0.442596
3,E,-1.244821


## INNER JOIN

In [ ]:
# SELECT *
# FROM df1
# INNER JOIN df2
#   ON df1.key = df2.key;

# merge performs an INNER JOIN by default; the intersection of the two tables (by the "key" value)
pd.merge(df1, df2, on="key")

## LEFT OUTER JOIN

In [5]:
# SELECT *
# FROM df1
# LEFT OUTER JOIN df2
#   ON df1.key = df2.key;

# merge/join first table with second, including ALL values from df1 with the intersecting values from df2
pd.merge(df1, df2, on="key", how="left")

,key,value_x,value_y
0,A,-1.908681,NaN
1,B,0.427489,-0.882033
2,C,0.352510,NaN
3,D,-0.455809,1.086580
4,D,-0.455809,2.343334


## RIGHT JOIN

In [38]:
# SELECT *
# FROM df1
# RIGHT OUTER JOIN df2
#   ON df1.key = df2.key;

# merge/join first table with second, including ALL values from DF2 with the intersecting values from DF1
pd.merge(df1, df2, on="key", how="right")

,key,value_x,value_y
0,B,1.476859,1.246087
1,D,-1.999917,0.267525
2,D,-1.999917,0.442596
3,E,NaN,-1.244821


## FULL JOIN

In [39]:
# SELECT *
# FROM df1
# FULL OUTER JOIN df2
#   ON df1.key = df2.key;

# all data from BOTH tables
pd.merge(df1, df2, on="key", how="outer")

,key,value_x,value_y
0,A,0.357361,NaN
1,B,1.476859,1.246087
2,C,-2.384841,NaN
3,D,-1.999917,0.267525
4,D,-1.999917,0.442596
5,E,NaN,-1.244821


# UNION

In [40]:
dfu1 = pd.DataFrame(
    {"city": ["Chicago", "San Francisco", "New York City"], "rank": range(1, 4)}
)


dfu2 = pd.DataFrame(
    {"city": ["Chicago", "Boston", "Los Angeles"], "rank": [1, 4, 5]}
)


In [ ]:
dfu1

In [ ]:
dfu2

In [45]:
pd.concat([dfu1, dfu2])

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
0,Chicago,1
1,Boston,4
2,Los Angeles,5


In [44]:
# SQL’s UNION is similar to UNION ALL, however UNION will remove duplicate rows.
# In pandas, you can use concat() in conjunction with drop_duplicates().
pd.concat([dfu1, dfu2]).drop_duplicates()

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
1,Boston,4
2,Los Angeles,5


# LIMIT

In [ ]:
# SELECT * FROM tips
# LIMIT 10;
tips.head(10)

## Top n rows with offset

In [ ]:
# SELECT * FROM tips
# ORDER BY tip DESC
# LIMIT 10 OFFSET 5;

tips.nlargest(10 + 5, columns="tip").tail(10)

## Top n rows per group¶

In [ ]:
# SELECT * FROM (
#   SELECT
#     t.*,
#     ROW_NUMBER() OVER(PARTITION BY day ORDER BY total_bill DESC) AS rn
#   FROM tips t
# )
# WHERE rn < 3
# ORDER BY day, rn;

(
    tips.assign(
        rn=tips.sort_values(["total_bill"], ascending=False)
        .groupby(["day"])
        .cumcount()
        + 1
    )
    .query("rn < 3")
    .sort_values(["day", "rn"])
)

# UPDATE

In [ ]:
# UPDATE tips
# SET tip = tip*2
# WHERE tip < 2;
tips.loc[tips["tip"] < 2, "tip"] *= 2

# DELETE

In [ ]:
# DELETE FROM tips
# WHERE tip > 9;
tips.loc[tips["tip"] <= 9]